In [1]:
import numpy

In [2]:
import sklearn

In [3]:
import survwrap

In [4]:
X, y = survwrap.load_test_data()
X.shape, y.shape

((198, 84), (198,))

### Generate a (stratified) train-test split and Scale the features (only) 

First do the stratified splitting THEN do scaling, parameterized on X_train set ONLY 

In [5]:
from sklearn.preprocessing import StandardScaler, RobustScaler

In [6]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y,
                                                                            stratify=survwrap.get_indicator(y), 
                                                                           random_state=2308)

In [7]:
scaler = StandardScaler().fit(X_train)
[X_train, X_test] = [ scaler.transform(_) for _ in  [X_train, X_test] ]
X_train.shape, X_test.shape

((148, 84), (50, 84))

In [8]:
#X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

balanced partitioning OK. Robst scaler damages the performance of DSM A LOT.
maybe did something wrong. It is standard scaler for now.

In [9]:
survwrap.get_indicator(y).sum(), survwrap.get_indicator(y_train).sum(), survwrap.get_indicator(y_test).sum(),


(51, 38, 13)

## check possible dimensionality reduction

In [10]:
from sklearn.decomposition import PCA

In [11]:
pca= PCA(n_components=0.995, random_state=2308).fit(X_train)
pca.n_components_

74

Only a modest dimensionality reduction is possible using PCA

In [12]:
## Stratified CV spliter for survival analysis

In [13]:
testkf= RepeatedStratifiedKFold(n_splits=5,n_repeats=2,random_state=2307)
for trn,tst in testkf.split(X_train, survwrap.get_indicator(y_train)):
    print(trn,tst) 

NameError: name 'RepeatedStratifiedKFold' is not defined

In [30]:
from sklearn.model_selection import RepeatedKFold, RepeatedStratifiedKFold

# test coxnet

In [23]:
coxnet = survwrap.CoxNet(rng_seed=2308)
coxnet.fit(X_train, y_train)

CoxNet(rng_seed=2308, l1_ratio=0.5, verbose=False, fit_baseline_model=False)

In [24]:
coxnet.score(X_test, y_test)

0.5330578512396694

In [25]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [27]:
def optimize(estimator, X, y, mode='sklearn-grid', user_grid=[] , cv=None):
    if mode == 'sklearn-grid':
        if not user_grid: 
            user_grid = estimator.get_parameter_grid()
        gs = GridSearchCV(estimator, user_grid, refit=True, cv=cv, n_jobs=3)
        gs.fit(X, y)
        return gs.best_estimator_, gs.best_params_, gs
    raise ValueError(f'unknown mode parameter: "{mode}"')

In [28]:
survwrap.CoxNet().get_parameter_grid()

{'l1_ratio': [0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99]}

In [31]:
opt_coxnet, opt_coxnet_params, opt_coxnet_search = optimize(survwrap.CoxNet(), X_train, y_train, user_grid={'l1_ratio':[0.01,0.25,0.5,0.75,0.8,0.99]},
                                                           cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=2308).split(X_train,survwrap.get_indicator(y_train)))
opt_coxnet.score(X_test, y_test), opt_coxnet_params

(0.5764462809917356, {'l1_ratio': 0.01})

In [32]:
def test_model(model_constructor, **model_params):
    m = model_constructor(**model_params, rng_seed=2308)
    m.fit(X_train, y_train)
    o = m.score(X_train, y_train)
    s = m.score(X_test, y_test)
    print(f'Train: {o: .4f} , Test: {s:.4f}')
    return s

# Test DSM

In [33]:
grid =survwrap.DeepSurvivalMachines.get_parameter_grid()
grid

{'n_distr': [2],
 'distr_kind': ['Weibull'],
 'batch_size': [16],
 'layer_sizes': [[10]],
 'learning_rate': [0.001],
 'validation_size': [0.1],
 'max_epochs': [10],
 'elbo': [False]}

In [34]:
test_model(survwrap.DeepSurvivalMachines, batch_size=16, layer_sizes=[16], n_distr=2, max_epochs=200)

  3%|███▉                                                                                                                              | 6/200 [00:00<00:06, 28.80it/s]

Train:  0.8134 , Test: 0.6178


0.6177685950413223

In [35]:
test_model(survwrap.DeepSurvivalMachines, batch_size=16, layer_sizes=[7,7,7], n_distr=2, max_epochs=200)

 10%|████████████▎                                                                                                                    | 19/200 [00:00<00:04, 41.99it/s]

Train:  0.8690 , Test: 0.6467


0.6466942148760331

In [36]:
test_model(survwrap.DeepSurvivalMachines, batch_size=16, layer_sizes=[5]*5, n_distr=2, max_epochs=200)

  9%|███████████▌                                                                                                                     | 18/200 [00:00<00:04, 37.44it/s]

Train:  0.7767 , Test: 0.6085


0.6084710743801653

In [128]:
test_model(survwrap.DeepSurvivalMachines,layer_sizes=[41], max_epochs=200)

  3%|█▋                                                        | 6/200 [00:00<00:08, 22.83it/s]

Train:  0.7555 , Test: 0.5971


0.5971074380165289

In [37]:
grid['n_distr']=[1,2]
some_primes = [3,5,7,11,13,17,19,23,29,31,37]
grid['layer_sizes']=[ [_] for _ in some_primes] + [ [_]*3 for _ in some_primes]
    #[31]*2,[29]*2,[27]*2, [25]*2, [23]*2, [21]*2, [19]*2, [17]*3, [15]*3, 
     #                [12]*4, [10]*5, [8]*6, [7]*7, ]
    #[40],[30,30], [30], [23,23], [13,13,13], [25],[11,11,11],[20],[9,9,9], 
                     #[15],[7,7,7],[10],[5,5,5]]
                     #[16],[14,14], [7,7,7]]
grid['max_epochs']=[200]
grid['batch_size']=[16]
grid

{'n_distr': [1, 2],
 'distr_kind': ['Weibull'],
 'batch_size': [16],
 'layer_sizes': [[3],
  [5],
  [7],
  [11],
  [13],
  [17],
  [19],
  [23],
  [29],
  [31],
  [37],
  [3, 3, 3],
  [5, 5, 5],
  [7, 7, 7],
  [11, 11, 11],
  [13, 13, 13],
  [17, 17, 17],
  [19, 19, 19],
  [23, 23, 23],
  [29, 29, 29],
  [31, 31, 31],
  [37, 37, 37]],
 'learning_rate': [0.001],
 'validation_size': [0.1],
 'max_epochs': [200],
 'elbo': [False]}

In [38]:
from sklearn.model_selection import RepeatedKFold 

In [142]:
# Non stratified CV
#opt_dsm, opt_dsm_params, opt_dsm_search = optimize(survwrap.DeepSurvivalMachines(rng_seed=2308),  X_train, y_train, 
#                                                   user_grid=grid,cv=RepeatedKFold(n_splits=5, n_repeats=3, random_state=2308))
#opt_dsm.score(X_test, y_test), opt_dsm_params

In [39]:
# Stratified CV
opt_dsm, opt_dsm_params, opt_dsm_search = optimize(survwrap.DeepSurvivalMachines(rng_seed=2308),  X_train, y_train, 
                                                   user_grid=grid,cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=2308).split(X_train,survwrap.get_indicator(y_train)))
opt_dsm.score(X_test, y_test), opt_dsm_params

  4%|█████▏                                                                                                                            | 8/200 [00:00<00:07, 27.25it/s]


(0.6838842975206612,
 {'batch_size': 16,
  'distr_kind': 'Weibull',
  'elbo': False,
  'layer_sizes': [13],
  'learning_rate': 0.001,
  'max_epochs': 200,
  'n_distr': 1,
  'validation_size': 0.1})

In [41]:
cv_res = opt_dsm_search.cv_results_
rez=sorted(zip(cv_res['rank_test_score'], cv_res['mean_test_score'] , cv_res['std_test_score'], 
               cv_res['mean_test_score'] - cv_res['std_test_score'], opt_dsm_search.cv_results_['params']))
rez[:10]

[(1,
  0.6800064902521725,
  0.08640148207431089,
  0.5936050081778617,
  {'batch_size': 16,
   'distr_kind': 'Weibull',
   'elbo': False,
   'layer_sizes': [13],
   'learning_rate': 0.001,
   'max_epochs': 200,
   'n_distr': 1,
   'validation_size': 0.1}),
 (2,
  0.6778373825259126,
  0.09205344136092578,
  0.5857839411649869,
  {'batch_size': 16,
   'distr_kind': 'Weibull',
   'elbo': False,
   'layer_sizes': [17],
   'learning_rate': 0.001,
   'max_epochs': 200,
   'n_distr': 2,
   'validation_size': 0.1}),
 (3,
  0.674360686971627,
  0.05712700290002999,
  0.617233684071597,
  {'batch_size': 16,
   'distr_kind': 'Weibull',
   'elbo': False,
   'layer_sizes': [37],
   'learning_rate': 0.001,
   'max_epochs': 200,
   'n_distr': 1,
   'validation_size': 0.1}),
 (4,
  0.6707957096968238,
  0.0607100595554704,
  0.6100856501413534,
  {'batch_size': 16,
   'distr_kind': 'Weibull',
   'elbo': False,
   'layer_sizes': [29],
   'learning_rate': 0.001,
   'max_epochs': 200,
   'n_distr': 2,


In [42]:
low_bound = sorted(rez[:10], key= lambda k: k[3], reverse=True)
for lb in [ _ for _ in low_bound if _[3] > 0.5] :
    print(lb[0:4])
    test_model(survwrap.DeepSurvivalMachines, **lb[4])

(3, 0.674360686971627, 0.05712700290002999, 0.617233684071597)


  3%|███▉                                                                                                                              | 6/200 [00:00<00:07, 24.69it/s]


Train:  0.8486 , Test: 0.6653
(6, 0.6683175258595094, 0.057869604945826984, 0.6104479209136825)


  5%|██████▍                                                                                                                          | 10/200 [00:00<00:05, 35.92it/s]


Train:  0.7811 , Test: 0.6198
(7, 0.6652983137465113, 0.05494023873549321, 0.610358075011018)


...

Train:  0.8032 , Test: 0.5992
(9, 0.6597070588850419, 0.08400503850693543, 0.5757020203781065)


  2%|█▉                                                                                                                                | 3/200 [00:00<00:11, 17.28it/s]


Train:  0.6866 , Test: 0.6343
(8, 0.6598382074175131, 0.09760034612352134, 0.5622378612939918)


  2%|██▌                                                                                                                               | 4/200 [00:00<00:09, 20.63it/s]


Train:  0.8286 , Test: 0.6550


In [43]:
test_model(survwrap.DeepSurvivalMachines, batch_size=16, layer_sizes=[13], n_distr=1, max_epochs=200)

  4%|█████▏                                                                                                                            | 8/200 [00:00<00:05, 36.70it/s]

Train:  0.8299 , Test: 0.6839


0.6838842975206612

# Appunti di varie prove DSM

## Insieme di prova (Breast cancer)

Piccolo insieme bastardo. 200 elementi, 84 features e solo il 25% di eventi non censurati.
Coxnet di default va male (c-index 53%), DSM richiede cura per inziare a generalizzare.

## Stabilità risultato

1. stratificare, stratificare rispetto all'evento. Usando insiemi stratificati rispetto all'evento DSM inizia a funzionare in maniera meno erratica.
2. Se si stratifica anche la CV, L'ottimizzazione DSM tende a fornire risultati buoni per tutta le soluzioni top e anche CoxNet ci guadagna in ottimizzazione (c-index: 57%).
   potrebbe sembrare ovvio a posteriori, dato che stiamo imponendo un "contenuto di informazione" simile tra train e test. ma non mi pare una forzatura clamorosa.

# Ottimizzazione parametri DSM

1. Quello più difficile da gestire è ovviamente la topologia della rete. Usando un solo strato nascosto si vede che anche cambiando di un solo valore la dimensione si può passare da una rete che generalizza a una che fa schifo, quindi bisognerebbe fare uno scan senza salti. O fare una random search.
   Alla fine mi sono inventato un protocollo scemo ma che sembra andare ragionevolmente (su breast cancer). Fare uno scan per i valori de numeri primi compresi tra 3 e n_features/2 (olè!)
2. Reti a "parametri equivalenti", (la procedura del quant): fatti tanti test, c'è un fenomeno curioso: le reti con numero pari di strarti nascosti generalizzano male. Mah!
   Quindi alla fine ho testato sistematicamente solo quelle con strati dispari (1,3,5). Ma per quelle con 5 iniziano ad apparire problemi di stabilità numerica, probabilmente quando sono troppo grandi.
3. numero di funzioni di base (Weibull): il default è 2, ma si ottengono parecchi buoni risultati anche con una sola. Molte soluzioni con 3 non generalizzano. quindi 1 o 2.
4. Come detto sopra, la stratificazione aiuta molto, così come il fare almeno 10 split (n_splits=5, n_repeats=2) per aver una varianza abbastanza stabile. Sono riuscito a fare griglie con cv stratificate con un trucchino sklearn che implementerei
5. batch size: 10% dell'insieme di train. Sembra OK. mai ottimizzata. così come gli altri parametri legati al learning. Non mi aspetto che incidano.
6. Alla fine della fiera si possono ottenere risulati stabilmente decorosi usando 1 o 3 strati nascosto e 1 sola weibull (test c-index: 65%). Introducendo anche la doppia weibull la griglia diventa più onerosa e possono aumentare le soluzioni solo apparentemente buone ma generalizzano peggio (anche se non disastrose: 58%). Non andare oltre le 2 weibull e 3 strati.
